In [1]:
import numpy as np
import pandas as pd
import data_als
import data_prepro
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [2]:
train,test = data_prepro.loadData('datasets/data_tr_city.csv','datasets/data_ts_city.csv',trainYearRange1=2019,trainYearRange2=2020)

In [3]:
train = data_prepro.outlierDataToNan(train,low=True,high=True,thres=10.0)

In [4]:
train = data_prepro.fillnaBehind(train)

Int64Index([17558, 21595, 25626], dtype='int64')
index                  0
datetime               0
구미 혁신도시배수지 유출유량 적산차    2
year                   0
month                  0
dtype: int64


In [5]:
train = data_prepro.fillprevValue(train)

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17544 entries, 0 to 17543
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                17544 non-null  int64  
 1   datetime             17544 non-null  object 
 2   구미 혁신도시배수지 유출유량 적산차  17544 non-null  float64
 3   year                 17544 non-null  int64  
 4   month                17544 non-null  int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 685.4+ KB


In [7]:
x_train,y_train = data_prepro.XDataToXAndYSeq(train,step=60,output=4)
x_test,y_test = data_prepro.XDataToXAndYSeq(test,step=60,output=4)

In [8]:
# y_train = y_train.reshape(-1,1,1)
# y_test = y_test.reshape(-1,1,1)

In [9]:
x_train = x_train.reshape(-1,60,1)
y_train = y_train.reshape(-1,4)

In [10]:
x_test = x_test.reshape(-1,60,1)
y_test = y_test.reshape(-1,4)

In [11]:
from torch.autograd import Variable 
import torch

c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
X_train_tensors = Variable(torch.Tensor(x_train))
X_test_tensors = Variable(torch.Tensor(x_test))

y_train_tensors = Variable(torch.Tensor(y_train))
y_test_tensors = Variable(torch.Tensor(y_test))

X_train_tensors_final = torch.reshape(X_train_tensors,   X_train_tensors.shape)
X_test_tensors_final = torch.reshape(X_test_tensors,  X_test_tensors.shape) 

print("Training Shape", X_train_tensors_final.shape, y_train_tensors.shape)
print("Testing Shape", X_test_tensors_final.shape, y_test_tensors.shape) 

Training Shape torch.Size([17480, 60, 1]) torch.Size([17480, 4])
Testing Shape torch.Size([8360, 60, 1]) torch.Size([8360, 4])


In [13]:
X_train_tensors_final[0].shape

torch.Size([60, 1])

In [14]:
X_train_tensors_final

tensor([[[114.],
         [ 83.],
         [ 68.],
         ...,
         [399.],
         [246.],
         [339.]],

        [[ 83.],
         [ 68.],
         [ 90.],
         ...,
         [246.],
         [339.],
         [364.]],

        [[ 68.],
         [ 90.],
         [112.],
         ...,
         [339.],
         [364.],
         [248.]],

        ...,

        [[ 46.],
         [ 59.],
         [ 33.],
         ...,
         [335.],
         [204.],
         [254.]],

        [[ 59.],
         [ 33.],
         [130.],
         ...,
         [204.],
         [254.],
         [473.]],

        [[ 33.],
         [130.],
         [459.],
         ...,
         [254.],
         [473.],
         [305.]]])

In [15]:
# train['Y'] = ExponentialSmoothing(train['구미 혁신도시배수지 유출유량 적산차'],
#                                  trend="add", # add || mul
#                                  damped_trend = True,                                 seasonal="add", # add || mul
#                                  seasonal_periods=24 # we set 12. It represents that 12 step (month for our case) equals a seasonal period
#                                  ).fit(
#                                   #  smoothing_level=0.1, # alpha
#                                   #       smoothing_trend=0.2, # beta
#                                   #       smoothing_seasonal=0.1 # gamma
#                                         ).fittedvalues

# # https://www.statsmodels.org/devel/generated/statsmodels.tsa.holtwinters.ExponentialSmoothing.html
# # https://www.kaggle.com/code/enesbaysan/applied-example-of-triple-exponential-smoothing

In [16]:
import torch.nn as nn

In [17]:
try:
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # device
except:
    device = "cpu"
print(torch.cuda.get_device_name(0))

Tesla M40 24GB


In [18]:
from torch.nn.utils import weight_norm

m = weight_norm(nn.Linear(20, 40), name='weight')
m # Linear(in_features=20, out_features=40, bias=True)
m.weight_g.size() # torch.Size([40, 1])
m.weight_v.size() # torch.Size([40, 20])

torch.Size([40, 20])

In [19]:
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        # chomp_size(padding size): (kernel_size-1) * dilation_size 
        self.chomp_size = chomp_size  

    def forward(self, x):
        # output: (N, C_in, L_in - chomp_size)
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        #--------------------- Dilated Causal Convolution --------------------- 
        '''
        input: (N, C_in, L_in) -> (N, C_out, L_in)
        output sequence의 길이는 변하지 않는다. 
        '''
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        #----------------------------------------------------------------------
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        #--------------------- Dilated Causal Convolution --------------------- 
        '''
        input: (N, C_in, L_in) -> output: (N, C_out, L_in)
        output sequence의 길이는 변하지 않는다. 
        '''
        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        #----------------------------------------------------------------------
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)
        
        # Dilated Causal Conv -> WeightedNorm -> ReLU -> Dropout -> ... (논문의 Residual block 구조와 동일)
        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        
        # Residual Connections 
        '''  
        만약 Dilated Causal Conv를 적용하기 전의 input channel과 적용한 후의 
        output channel이 달라질 경우를 대비하여 추가적인 1*1 convolution을 추가해줌.

        input: (N, C_in, L_in) -> output: (N, C_out, L_in)
        output sequence의 길이는 변하지 않는다. (1*1 convolution)
        '''  
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            # dilation factor는 layer의 깊이에 지수적으로 증가 (2^i, i: layer depth) 
            dilation_size = 2 ** i 
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            # 설정된 layer의 깊이만큼 TemporalBlock 생성 
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)


class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout):
        super(TCN, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size=kernel_size, dropout=dropout)
        self.linear = nn.Linear(num_channels[-1], output_size)

    def forward(self, inputs):
        """
        Inputs have to have dimension (N, C_in, L_in)
        Flatten된 MNIST의 경우에는 (batch_size, 1, 784)
        """
        y1 = self.tcn(inputs)  # input should have dimension (N, C, L)
        o = self.linear(y1[:, :, -1]) # sequence의 마지막 time step로 linear계산 -> 분류예측
        return o

In [37]:
num_epochs = 100000 #1000 epochs
learning_rate = 0.01 #0.001 lr

kernel_size = 5
stride = 1
dilation = 1
padding=2
dropout=0.2
input_size = 60 #number of features
hidden_size = 256 #number of features in hidden state
num_channels = [256,256,128] #number of stacked lstm layers

output_size = 4 #number of output classes 

In [21]:
tcn = TCN(input_size, output_size, num_channels, kernel_size, dropout).to(device)

loss_function = torch.nn.L1Loss()    # mean-squared error for regression
optimizer = torch.optim.Adam(tcn.parameters(), lr=learning_rate)  # adam optimizer

In [38]:
tcn.to("cuda")

TCN(
  (tcn): TemporalConvNet(
    (network): Sequential(
      (0): TemporalBlock(
        (conv1): Conv1d(60, 256, kernel_size=(5,), stride=(1,), padding=(4,))
        (chomp1): Chomp1d()
        (relu1): ReLU()
        (dropout1): Dropout(p=0.2, inplace=False)
        (conv2): Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(4,))
        (chomp2): Chomp1d()
        (relu2): ReLU()
        (dropout2): Dropout(p=0.2, inplace=False)
        (net): Sequential(
          (0): Conv1d(60, 256, kernel_size=(5,), stride=(1,), padding=(4,))
          (1): Chomp1d()
          (2): ReLU()
          (3): Dropout(p=0.2, inplace=False)
          (4): Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(4,))
          (5): Chomp1d()
          (6): ReLU()
          (7): Dropout(p=0.2, inplace=False)
        )
        (downsample): Conv1d(60, 256, kernel_size=(1,), stride=(1,))
        (relu): ReLU()
      )
      (1): TemporalBlock(
        (conv1): Conv1d(256, 256, kernel_size=(5,), str

In [39]:
for epoch in range(num_epochs):
    outputs = tcn.forward(X_train_tensors_final.to(device)) #forward pass
    
    # obtain the loss function
    loss = loss_function(outputs, y_train_tensors.to(device))
    optimizer.zero_grad() #caluclate the gradient, manually setting to 0
    loss.backward() #calculates the loss of the loss function
    optimizer.step()
    if epoch%20 == 0:
        
        outputs = tcn.forward(X_test_tensors_final.to(device)) #forward pass
        test_loss = loss_function(outputs, y_test_tensors.to(device))
        # loss.backward() #calculates the loss of the loss function
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()),end=', ')
        print("test data loss",test_loss.item())

Epoch: 0, loss: 138.12569, test data loss 139.873779296875
Epoch: 20, loss: 138.12569, test data loss 139.873779296875
Epoch: 40, loss: 138.12572, test data loss 139.87368774414062
Epoch: 60, loss: 138.12566, test data loss 139.87368774414062
Epoch: 80, loss: 138.12561, test data loss 139.8735809326172
Epoch: 100, loss: 138.12561, test data loss 139.87374877929688
Epoch: 120, loss: 138.12564, test data loss 139.87368774414062
Epoch: 140, loss: 138.12569, test data loss 139.8737030029297
Epoch: 160, loss: 138.12569, test data loss 139.8736572265625
Epoch: 180, loss: 138.12575, test data loss 139.87368774414062
Epoch: 200, loss: 138.12564, test data loss 139.87374877929688
Epoch: 220, loss: 138.12575, test data loss 139.8737030029297
Epoch: 240, loss: 138.12569, test data loss 139.8737030029297
Epoch: 260, loss: 138.12572, test data loss 139.87359619140625
Epoch: 280, loss: 138.12561, test data loss 139.87379455566406
Epoch: 300, loss: 138.12566, test data loss 139.87376403808594
Epoch: 

In [40]:
torch.save(tcn.state_dict(), 'tcn_100000epochs.pt')

In [41]:
tcn.to("cpu")

TCN(
  (tcn): TemporalConvNet(
    (network): Sequential(
      (0): TemporalBlock(
        (conv1): Conv1d(60, 256, kernel_size=(5,), stride=(1,), padding=(4,))
        (chomp1): Chomp1d()
        (relu1): ReLU()
        (dropout1): Dropout(p=0.2, inplace=False)
        (conv2): Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(4,))
        (chomp2): Chomp1d()
        (relu2): ReLU()
        (dropout2): Dropout(p=0.2, inplace=False)
        (net): Sequential(
          (0): Conv1d(60, 256, kernel_size=(5,), stride=(1,), padding=(4,))
          (1): Chomp1d()
          (2): ReLU()
          (3): Dropout(p=0.2, inplace=False)
          (4): Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(4,))
          (5): Chomp1d()
          (6): ReLU()
          (7): Dropout(p=0.2, inplace=False)
        )
        (downsample): Conv1d(60, 256, kernel_size=(1,), stride=(1,))
        (relu): ReLU()
      )
      (1): TemporalBlock(
        (conv1): Conv1d(256, 256, kernel_size=(5,), str

In [42]:
X_train_tensors_final[0].shape

torch.Size([60, 1])

In [43]:
tcn(X_train_tensors_final[0].reshape(1,60,1))

tensor([[235.8342, 235.8221, 235.8173, 235.8231]], grad_fn=<AddmmBackward0>)